LLM weights is usually not enough to give accurate and insightful answers to our questions thus we need external tools to access the better functionality and web/world.

In [40]:
import os
import re
from groq import Groq
from dotenv import load_dotenv
import requests
import json

In [100]:

def get_weather(city_name: str):
    """
    Fetches weather data for a given city using OpenWeatherMap API.

    Args:
        city_name (str): Name of the city (e.g., "London").

    Returns:
        dict: Weather data including temperature, weather condition, etc.
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city_name,
        'appid': "157b97604b50157e6df56194df6f4d94",
        'units': 'metric'  # use 'imperial' for Fahrenheit
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'wind_speed': data['wind']['speed']
        }
        return weather
    else:
        return {'error': f"Failed to get weather data: {response.status_code}"}


In [42]:
city = "Delhi"
weather_info = get_weather(city)

if 'error' in weather_info:
    print(weather_info['error'])
else:
    weather_report = {
    "city": weather_info['city'],
    "temperature": f"{weather_info['temperature']}°C",
    "condition": weather_info['description'],
    "humidity": f"{weather_info['humidity']}%",
    "wind_speed": f"{weather_info['wind_speed']} m/s"
    }
    print(weather_report)
    


{'city': 'Delhi', 'temperature': '31.46°C', 'condition': 'few clouds', 'humidity': '41%', 'wind_speed': '6.13 m/s'}


In [43]:
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return JSON with function name and arguments as follows:

{"name": <function-name>,"arguments": <args-dict>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city will be mentioned by user"
}
</tools>
"""

In [44]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]

agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current weather in Delhi?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)
dict_output = json.loads(output)
type(dict_output)

{"name": "get_current_weather","arguments": {"location": "Delhi"}}


dict

In [45]:
result = get_weather(**dict_output["arguments"])
# print(result)
# type(result)
str_op = json.dumps(result)
type(str_op)
print(str_op)

{"city": "Delhi", "temperature": 31.46, "description": "few clouds", "humidity": 41, "wind_speed": 6.13}


In [46]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [47]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current weather in Delhi is mostly sunny with a few clouds, and the temperature is around 31.46°C (88.63°F). The humidity is relatively moderate at 41%, and there is a gentle breeze with a wind speed of 6.13 meters per second (13.7 miles per hour).'

**Testing the Tool Agent class**

In [69]:
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

In [70]:
weather_tool = tool(get_weather)


In [71]:
weather_tool.name

'get_weather'

In [72]:
json.loads(weather_tool.fn_signature)

{'name': 'get_weather',
 'description': '\n    Fetches weather data for a given city using OpenWeatherMap API.\n\n    Args:\n        city_name (str): Name of the city (e.g., "London").\n\n    Returns:\n        dict: Weather data including temperature, weather condition, etc.\n    ',
 'parameters': {'properties': {}}}

Let's create the agent tool now

In [73]:
tool_agent = ToolAgent(tools=[weather_tool])


Checking the tool flow, if we ask something absurd i.e. not related to tool it should not call the tool.

In [74]:
output = tool_agent.run(user_msg="just check")
output


"It seems like you didn't provide any specific information or task for me to check. If you could provide more context or clarify what you need me to check, I'll do my best to assist you."

In [ ]:
output = tool_agent.run(user_msg="weather at delhi")
print(output)